In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pickle

import tensorflow as tf
from keras import Sequential, layers, optimizers, models
from random import randint

import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output

#* for small network, using GPU will not provide a lot of speed improvement -> the following line limits the machine to use only CPU instead
tf.config.set_visible_devices([], 'GPU')

__Acknowledgement__

[Tech With Tim: Machine Learning Tutorial](https://www.youtube.com/watch?v=ujTCoH21GlA&list=PLzMcBGfZo4-mP7qA9cagf68V06sko5otr) (Youtube)

Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Thompson JJ, Blair MR, Chen L, Henrey AJ (2013) Video Game Telemetry as a Critical Tool in the Study of Complex Skill Learning. PLoS ONE 8(9): e75129.

# Data Processing

data from UCI Machine Learning Repositories: https://archive.ics.uci.edu/ml/datasets/SkillCraft1+Master+Table+Dataset

In [2]:
data = pd.read_csv('../data/SkillCraft1_Dataset.csv', na_values='?')
data = data.drop(['GameID'], axis=1)

filtered_data = data[data[data.columns].notnull().all(1)] # filter out any row that contains missing value
filtered_data

,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
0,5,27.0,10.0,3000.0,143.7180,0.003515,0.000220,7,0.000110,0.000392,0.004849,32.6677,40.8673,4.7508,28,0.001397,6,0.0,0.000000
1,5,23.0,10.0,5000.0,129.2322,0.003304,0.000259,4,0.000294,0.000432,0.004307,32.9194,42.3454,4.8434,22,0.001193,5,0.0,0.000208
2,4,30.0,10.0,200.0,69.9612,0.001101,0.000336,4,0.000294,0.000461,0.002926,44.6475,75.3548,4.0430,22,0.000745,6,0.0,0.000189
3,3,19.0,20.0,400.0,107.6016,0.001034,0.000213,1,0.000053,0.000543,0.003783,29.2203,53.7352,4.9155,19,0.000426,7,0.0,0.000384
4,3,32.0,10.0,500.0,122.8908,0.001136,0.000327,2,0.000000,0.001329,0.002368,22.6885,62.0813,9.3740,15,0.001174,4,0.0,0.000019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3334,4,20.0,8.0,400.0,158.1390,0.013829,0.000504,7,0.000217,0.000313,0.003583,36.3990,66.2718,4.5097,30,0.001035,7,0.0,0.000287
3335,5,16.0,56.0,1500.0,186.1320,0.006951,0.000360,6,0.000083,0.000166,0.005414,22.8615,34.7417,4.9309,38,0.001343,7,0.0,0.000388
3336,4,21.0,8.0,100.0,121.6992,0.002956,0.000241,8,0.000055,0.000208,0.003690,35.5833,57.9585,5.4154,23,0.002014,7,0.0,0.000000
3337,3,20.0,28.0,400.0,134.2848,0.005424,0.000182,5,0.000000,0.000480,0.003205,18.2927,62.4615,6.0202,18,0.000934,5,0.0,0.000000


# Visualization Tool

In [3]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                options=list(filtered_data.columns),
                value='LeagueIndex',
                id='y-axis'),
            dcc.RadioItems(
                options=['linear', 'log'],
                value='linear',
                id='y-axis-type',
                inline=True)], 
            style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Dropdown(
                options=list(filtered_data.columns),
                value='APM',
                id='x-axis'),
            dcc.RadioItems(
                options=['linear', 'log'],
                value='linear',
                id='x-axis-type',
                inline=True)],
            style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    dcc.Graph(id='graph', figure={}),
])

@app.callback(
    Output('graph', 'figure'),
    Input('y-axis', 'value'),
    Input('y-axis-type', 'value'),
    Input('x-axis', 'value'),
    Input('x-axis-type', 'value'))
def update_graph(y_axis, y_axis_type, x_axis, x_axis_type):
    fig = px.scatter(
        filtered_data,
        y=y_axis if y_axis_type == 'linear' else np.log10(filtered_data[y_axis]),
        x=x_axis if x_axis_type == 'linear' else np.log10(filtered_data[x_axis]),
        color='LeagueIndex',
        color_continuous_scale=px.colors.sequential.Rainbow,
        title="Custom Graph Generator")
    return fig

# Run app and display result inline in the notebook
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



# Linear Model

In [4]:
predict = 'LeagueIndex'

x = np.array(filtered_data.drop([predict], axis=1))
y = np.array(filtered_data[predict])

print('Model Construction\n------------------')
best_acc = 0 
for _ in range(100):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)
    acc = linear.score(x_test, y_test)
    if acc > best_acc:
        best_acc = acc
        with open('../model/8LeagueSkills_LinearModel.pickle', 'wb') as f:
            pickle.dump(linear, f)
        print(f'New most accurate model ({best_acc}) is saved!')

Model Construction
------------------
New most accurate model (0.5535592465027166) is saved!
New most accurate model (0.6142984954943991) is saved!


__Prediction__

In [5]:
model = open('../model/8LeagueSkills_LinearModel.pickle', 'rb')
linear: linear_model.LinearRegression = pickle.load(model)

In [6]:
print('\nPredition\n---------')
rank: dict = {1: 'Bronze', 2: 'Silver', 3: 'Gold', 4: 'Platinum', 5: 'Diamond', 6: 'Master', 7: 'GrandMaster', 8: 'Professional'}

predictions = linear.predict(x_test)
for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction.round(0)]}, Actual: {rank[y_test[i].round(0)]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i].round(0)]}')
    else:
        break


Predition
---------
Prediction: Diamond, Actual: Diamond
Prediction: Platinum, Actual: Platinum
Prediction: Platinum, Actual: Gold
Prediction: Diamond, Actual: Diamond
Prediction: Diamond, Actual: Master
Prediction: Platinum, Actual: Platinum
Prediction: Gold, Actual: Silver
Prediction: Platinum, Actual: Platinum
Prediction: Diamond, Actual: Diamond
Prediction: Diamond, Actual: Master


## 4 Leagues Categorization

In [7]:
fourLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver', 2: 'Gold-Platinum', 3: 'Diamond-Master', 4: 'GrandMaster-Professional'}
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 2
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 2
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 3
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 4
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 4
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 4

predict = 'LeagueIndex'

x = np.array(fourLeague_data.drop([predict], axis=1))
y = np.array(fourLeague_data[predict])

print('Model Construction\n------------------')
best_acc = 0 
for _ in range(100):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)
    acc = linear.score(x_test, y_test)
    if acc > best_acc:
        best_acc = acc
        with open('../model/4LeagueSkills_LinearModel.pickle', 'wb') as f:
            pickle.dump(linear, f)
        print(f'New most accurate model ({best_acc}) is saved!')

print('\nPredition\n---------')
predictions = linear.predict(x_test)
for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction.round(0)]}, Actual: {rank[y_test[i].round(0)]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i].round(0)]}')
    else:
        break

Model Construction
------------------
New most accurate model (0.4765101352373652) is saved!
New most accurate model (0.5134134595300379) is saved!
New most accurate model (0.5344179760862637) is saved!
New most accurate model (0.6138972156758241) is saved!

Predition
---------
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: GrandMaster-Professional, Actual: GrandMaster-Professional
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: Gold-Platinum, Actual: GrandMaster-Professional
Prediction: GrandMaster-Professional, Actual: GrandMaster-Professional
Prediction: GrandMaster-Professional, Actual: GrandMaster-Professional
Prediction: Gold-Platinum, Actual: Diamond-Master


## 3 Leagues Categorization

In [8]:
threeLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver-Gold', 2: 'Platinum-Diamond-Master', 3: 'GrandMaster-Professional'}
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 1
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 3
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 3

predict = 'LeagueIndex'

x = np.array(threeLeague_data.drop([predict], axis=1))
y = np.array(threeLeague_data[predict])

print('Model Construction\n------------------')
best_acc = 0 
for _ in range(100):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)
    acc = linear.score(x_test, y_test)
    if acc > best_acc:
        best_acc = acc
        with open('../model/3LeagueSkills_LinearModel.pickle', 'wb') as f:
            pickle.dump(linear, f)
        print(f'New most accurate model ({best_acc}) is saved!')

print('\nPredition\n---------')
predictions = linear.predict(x_test)
for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction.round(0)]}, Actual: {rank[y_test[i].round(0)]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i].round(0)]}')
    else:
        break

Model Construction
------------------
New most accurate model (0.4608866533546786) is saved!

Predition
---------
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master


## 2 Leagues Categorization

In [9]:
twoLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver-Gold-Platinum', 2: 'Diamond-Master-GrandMaster-Professional'}
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 2

predict = 'LeagueIndex'

x = np.array(twoLeague_data.drop([predict], axis=1))
y = np.array(twoLeague_data[predict])

print('Model Construction\n------------------')
best_acc = 0 
for _ in range(100):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)
    acc = linear.score(x_test, y_test)
    if acc > best_acc:
        best_acc = acc
        with open('../model/2LeagueSkills_LinearModel.pickle', 'wb') as f:
            pickle.dump(linear, f)
        print(f'New most accurate model ({best_acc}) is saved!')

print('\nPredition\n---------')
predictions = linear.predict(x_test)
for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction.round(0)]}, Actual: {rank[y_test[i].round(0)]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i].round(0)]}')
    else:
        break

Model Construction
------------------
New most accurate model (0.41939294090028845) is saved!
New most accurate model (0.42873973199680837) is saved!
New most accurate model (0.46028821473446846) is saved!

Predition
---------
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Bronze-Silver-Gold-Platinum, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-S

# Neighbor Model

In [10]:
predict = 'LeagueIndex'

x = np.array(filtered_data.drop([predict], axis=1))
y = np.array(filtered_data[predict])

print('Model Construction\n------------------')
best_acc = 0
for _ in range(10):
    for k in range(3, 14, 2):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

        model = KNeighborsClassifier(n_neighbors=k)
        
        model.fit(x_train, y_train)
        acc = model.score(x_test, y_test)
        if acc > best_acc:
            best_acc = acc
            with open('../model/8LeagueSkills_KNearestNeighborModel.pickle', 'wb') as f:
                pickle.dump(model, f)
                print(f'New most accurate model ({best_acc}) using {k} neighbors is saved!')

Model Construction
------------------
New most accurate model (0.3143712574850299) using 3 neighbors is saved!
New most accurate model (0.3532934131736527) using 5 neighbors is saved!
New most accurate model (0.40718562874251496) using 11 neighbors is saved!


__Prediction__

In [11]:
model = open('../model/8LeagueSkills_KNearestNeighborModel.pickle', 'rb')
model: KNeighborsClassifier = pickle.load(model)

In [12]:
print('\nPredition\n---------')
rank: dict = {1: 'Bronze', 2: 'Silver', 3: 'Gold', 4: 'Platinum', 5: 'Diamond', 6: 'Master', 7: 'GrandMaster', 8: 'Professional'}

predictions = model.predict(x_test)

for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction]}, Actual: {rank[y_test[i]]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i]]}')
    else:
        break


Predition
---------
Prediction: Master, Actual: Master
Prediction: Bronze, Actual: Silver
Prediction: Diamond, Actual: Diamond
Prediction: Platinum, Actual: Gold
Prediction: Silver, Actual: Gold
Prediction: Diamond, Actual: Master
Prediction: Silver, Actual: Bronze
Prediction: Gold, Actual: Platinum
Prediction: Master, Actual: Platinum
Prediction: Platinum, Actual: Silver


## 4 Leagues Categorization

In [13]:
fourLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver', 2: 'Gold-Platinum', 3: 'Diamond-Master', 4: 'GrandMaster-Professional'}
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 2
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 2
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 3
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 4
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 4
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 4

predict = 'LeagueIndex'

x = np.array(fourLeague_data.drop([predict], axis=1))
y = np.array(fourLeague_data[predict])

print('Model Construction\n------------------')
best_acc = 0
for _ in range(10):
    for k in range(3, 14, 2):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

        model = KNeighborsClassifier(n_neighbors=k)
        
        model.fit(x_train, y_train)
        acc = model.score(x_test, y_test)
        if acc > best_acc:
            best_acc = acc
            with open('../model/4LeagueSkills_KNearestNeighborModel.pickle', 'wb') as f:
                pickle.dump(model, f)
                print(f'New most accurate model ({best_acc}) using {k} neighbors is saved!')

print('\nPredition\n---------')
predictions = model.predict(x_test)

for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction]}, Actual: {rank[y_test[i]]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i]]}')
    else:
        break

Model Construction
------------------
New most accurate model (0.46706586826347307) using 3 neighbors is saved!
New most accurate model (0.49700598802395207) using 5 neighbors is saved!
New most accurate model (0.5538922155688623) using 13 neighbors is saved!
New most accurate model (0.5658682634730539) using 9 neighbors is saved!

Predition
---------
Prediction: Diamond-Master, Actual: Gold-Platinum
Prediction: Diamond-Master, Actual: Gold-Platinum
Prediction: Gold-Platinum, Actual: Bronze-Silver
Prediction: Diamond-Master, Actual: GrandMaster-Professional
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: Bronze-Silver, Actual: Gold-Platinum
Prediction: Gold-Platinum, Actual: Diamond-Master
Prediction: Bronze-Silver, Actual: Bronze-Silver
Prediction: Gold-Platinum, Actual: Gold-Platinum
Prediction: Bronze-Silver, Actual: Bronze-Silver


## 3 Leagues Categorization

In [14]:
threeLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver-Gold', 2: 'Platinum-Diamond-Master', 3: 'GrandMaster-Professional'}
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 1
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 3
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 3

predict = 'LeagueIndex'

x = np.array(threeLeague_data.drop([predict], axis=1))
y = np.array(threeLeague_data[predict])

print('Model Construction\n------------------')
best_acc = 0
for _ in range(10):
    for k in range(3, 14, 2):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

        model = KNeighborsClassifier(n_neighbors=k)
        
        model.fit(x_train, y_train)
        acc = model.score(x_test, y_test)
        if acc > best_acc:
            best_acc = acc
            with open('../model/3LeagueSkills_KNearestNeighborModel.pickle', 'wb') as f:
                pickle.dump(model, f)
                print(f'New most accurate model ({best_acc}) using {k} neighbors is saved!')

print('\nPredition\n---------')
predictions = model.predict(x_test)

for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction]}, Actual: {rank[y_test[i]]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i]]}')
    else:
        break

Model Construction
------------------
New most accurate model (0.7365269461077845) using 3 neighbors is saved!
New most accurate model (0.7574850299401198) using 5 neighbors is saved!
New most accurate model (0.7904191616766467) using 7 neighbors is saved!
New most accurate model (0.7964071856287425) using 13 neighbors is saved!
New most accurate model (0.7994011976047904) using 7 neighbors is saved!
New most accurate model (0.844311377245509) using 7 neighbors is saved!

Predition
---------
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Bronze-Silver-Gold
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master

## 2 Leagues Categorization

In [15]:
twoLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver-Gold-Platinum', 2: 'Diamond-Master-GrandMaster-Professional'}
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 2

predict = 'LeagueIndex'

x = np.array(twoLeague_data.drop([predict], axis=1))
y = np.array(twoLeague_data[predict])

print('Model Construction\n------------------')
best_acc = 0
for _ in range(10):
    for k in range(3, 14, 2):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

        model = KNeighborsClassifier(n_neighbors=k)

        model.fit(x_train, y_train)
        acc = model.score(x_test, y_test)
        if acc > best_acc:
            best_acc = acc
            with open('../model/3LeagueSkills_KNearestNeighborModel.pickle', 'wb') as f:
                pickle.dump(model, f)
                print(f'New most accurate model ({best_acc}) using {k} neighbors is saved!')

print('\nPredition\n---------')
predictions = model.predict(x_test)

for i, prediction in enumerate(predictions):
    if i < 10:
        try:
            print(f'Prediction: {rank[prediction]}, Actual: {rank[y_test[i]]}')
        except KeyError:
            print(f'Prediction: Unknown, Actual: {rank[y_test[i]]}')
    else:
        break

Model Construction
------------------
New most accurate model (0.7754491017964071) using 3 neighbors is saved!
New most accurate model (0.8083832335329342) using 5 neighbors is saved!
New most accurate model (0.811377245508982) using 11 neighbors is saved!
New most accurate model (0.8143712574850299) using 13 neighbors is saved!

Predition
---------
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platin

# Neural Network

In [16]:
predict = 'LeagueIndex'
rank: dict = {1: 'Bronze', 2: 'Silver', 3: 'Gold', 4: 'Platinum', 5: 'Diamond', 6: 'Master', 7: 'GrandMaster', 8: 'Professional'}

x = np.array(filtered_data.drop([predict], axis=1))
y = np.array(filtered_data[predict])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [17]:
model = Sequential()
model.add(layers.Dense(16, input_shape=(18,), activation='sigmoid'))
model.add(layers.Dense(8, activation='sigmoid'))
model.add(layers.Dense(8, activation='softmax')) #? Dense(8) works
model.summary()

#! using optimizer = 'adam' does not work for M1 architecture
model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('../model/8LeagueSkills_NeuralNetwork.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                304       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
Total params: 512
Trainable params: 512
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2023-05-07 20:36:00.122487: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


94/94 [==============================] - 0s 630us/step - loss: 2.1726 - accuracy: 0.1245
Epoch 2/10
94/94 [==============================] - 0s 503us/step - loss: 1.9471 - accuracy: 0.2468
Epoch 3/10
94/94 [==============================] - 0s 507us/step - loss: 1.8191 - accuracy: 0.2601
Epoch 4/10
94/94 [==============================] - 0s 490us/step - loss: 1.7435 - accuracy: 0.2804
Epoch 5/10
94/94 [==============================] - 0s 493us/step - loss: 1.6985 - accuracy: 0.3054
Epoch 6/10
94/94 [==============================] - 0s 488us/step - loss: 1.6648 - accuracy: 0.3120
Epoch 7/10
94/94 [==============================] - 0s 482us/step - loss: 1.6332 - accuracy: 0.3114
Epoch 8/10
94/94 [==============================] - 0s 480us/step - loss: 1.6030 - accuracy: 0.3160
Epoch 9/10
94/94 [==============================] - 0s 491us/step - loss: 1.5738 - accuracy: 0.3243
Epoch 10/10
11/11 [==============================] - 0s 670us/step - loss: 1.5198 - accuracy: 0.3084
Test accur

__Prediction__

In [18]:
model: Sequential = models.load_model('../model/8LeagueSkills_NeuralNetwork.h5')
predictions = model.predict(x_train)

for _ in range(10):
    i = randint(0, len(predictions))
    print(f'Prediction: {rank[np.argmax(predictions[i])]}, Actual: {rank[y_train[i]]}')

94/94 [==============================] - 0s 401us/step
Prediction: Diamond, Actual: Platinum
Prediction: Gold, Actual: Bronze
Prediction: Gold, Actual: Silver
Prediction: Platinum, Actual: Platinum
Prediction: Diamond, Actual: Diamond
Prediction: Platinum, Actual: Platinum
Prediction: Gold, Actual: Silver
Prediction: Gold, Actual: Gold
Prediction: Gold, Actual: Silver
Prediction: Diamond, Actual: Diamond


## 4 Leagues Prediction

In [19]:
fourLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver', 2: 'Gold-Platinum', 3: 'Diamond-Master', 4: 'GrandMaster-Professional'}
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 2
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 2
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 3
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 4
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 4
fourLeague_data.loc[fourLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 4

predict = 'LeagueIndex'

x = np.array(fourLeague_data.drop([predict], axis=1))
y = np.array(fourLeague_data[predict])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

model = Sequential()
model.add(layers.Dense(16, input_shape=(18,), activation='sigmoid'))
model.add(layers.Dense(8, activation='sigmoid'))
model.add(layers.Dense(5, activation='softmax')) #? Dense(4) doesn't work
model.summary()

#! using optimizer = 'adam' does not work for M1 architecture
model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('../model/4LeagueSkills_NeuralNetwork.h5')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                304       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 5)                 45        
                                                                 
Total params: 485
Trainable params: 485
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
94/94 [==============================] - 0s 891us/step - loss: 1.5531 - accuracy: 0.4063
Epoch 2/10
94/94 [==============================] - 0s 738us/step - loss: 1.3930 - accuracy: 0.4089
Epoch 3/10
94/94 [==============================] - 0s 737us/step - loss: 1.3261 - accuracy: 0.4089
Epoch 4/10
94/94 [==

In [20]:
predictions = model.predict(x_train)

for _ in range(10):
    i = randint(0, len(predictions))
    print(f'Prediction: {rank[np.argmax(predictions[i])]}, Actual: {rank[y_train[i]]}')

94/94 [==============================] - 0s 419us/step
Prediction: Gold-Platinum, Actual: Diamond-Master
Prediction: Gold-Platinum, Actual: Diamond-Master
Prediction: GrandMaster-Professional, Actual: GrandMaster-Professional
Prediction: GrandMaster-Professional, Actual: Diamond-Master
Prediction: Gold-Platinum, Actual: Bronze-Silver
Prediction: GrandMaster-Professional, Actual: GrandMaster-Professional
Prediction: Gold-Platinum, Actual: Bronze-Silver
Prediction: Gold-Platinum, Actual: Diamond-Master
Prediction: Gold-Platinum, Actual: GrandMaster-Professional
Prediction: Gold-Platinum, Actual: Gold-Platinum


## 3 Leagues Prediction

In [21]:
threeLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver-Gold', 2: 'Platinum-Diamond-Master', 3: 'GrandMaster-Professional'}
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 1
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 2
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 3
threeLeague_data.loc[threeLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 3

predict = 'LeagueIndex'

x = np.array(threeLeague_data.drop([predict], axis=1))
y = np.array(threeLeague_data[predict])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

model = Sequential()
model.add(layers.Dense(16, input_shape=(18,), activation='sigmoid'))
model.add(layers.Dense(8, activation='sigmoid'))
model.add(layers.Dense(4, activation='softmax')) #? Dense(3) doesn't work
model.summary()

#! using optimizer = 'adam' does not work for M1 architecture
model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('../model/3LeagueSkills_NeuralNetwork.h5')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                304       
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dense_8 (Dense)             (None, 4)                 36        
                                                                 
Total params: 476
Trainable params: 476
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
94/94 [==============================] - 0s 605us/step - loss: 1.2693 - accuracy: 0.4529
Epoch 2/10
94/94 [==============================] - 0s 524us/step - loss: 0.8888 - accuracy: 0.7263
Epoch 3/10
94/94 [==============================] - 0s 561us/step - loss: 0.7187 - accuracy: 0.7239
Epoch 4/10
94/94 [==

In [22]:
predictions = model.predict(x_train)

for _ in range(10):
    i = randint(0, len(predictions))
    print(f'Prediction: {rank[np.argmax(predictions[i])]}, Actual: {rank[y_train[i]]}')

94/94 [==============================] - 0s 422us/step
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Platinum-Diamond-Master, Actual: GrandMaster-Professional
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master
Prediction: Bronze-Silver-Gold, Actual: Platinum-Diamond-Master
Prediction: Bronze-Silver-Gold, Actual: Bronze-Silver-Gold
Prediction: Platinum-Diamond-Master, Actual: Platinum-Diamond-Master


## 2 Leagues Prediction

In [23]:
twoLeague_data = filtered_data.copy()

rank: dict = {1: 'Bronze-Silver-Gold-Platinum', 2: 'Diamond-Master-GrandMaster-Professional'}
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 2, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 3, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 4, 'LeagueIndex'] = 1
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 5, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 6, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 7, 'LeagueIndex'] = 2
twoLeague_data.loc[twoLeague_data['LeagueIndex'] == 8, 'LeagueIndex'] = 2

predict = 'LeagueIndex'

x = np.array(twoLeague_data.drop([predict], axis=1))
y = np.array(twoLeague_data[predict])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

model = Sequential()
model.add(layers.Dense(16, input_shape=(18,), activation='sigmoid'))
model.add(layers.Dense(8, activation='sigmoid'))
model.add(layers.Dense(3, activation='softmax')) #? Dense(2) doesn't work
model.summary()

#! using optimizer = 'adam' does not work for M1 architecture
model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

model.save('../model/2LeagueSkills_NeuralNetwork.h5')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 16)                304       
                                                                 
 dense_10 (Dense)            (None, 8)                 136       
                                                                 
 dense_11 (Dense)            (None, 3)                 27        
                                                                 
Total params: 467
Trainable params: 467
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
94/94 [==============================] - 0s 596us/step - loss: 0.8448 - accuracy: 0.5338
Epoch 2/10
94/94 [==============================] - 0s 511us/step - loss: 0.7379 - accuracy: 0.5611
Epoch 3/10
94/94 [==============================] - 0s 473us/step - loss: 0.6905 - accuracy: 0.5611
Epoch 4/10
94/94 [==

In [24]:
predictions = model.predict(x_train)

for _ in range(10):
    i = randint(0, len(predictions))
    print(f'Prediction: {rank[np.argmax(predictions[i])]}, Actual: {rank[y_train[i]]}')

94/94 [==============================] - 0s 394us/step
Prediction: Bronze-Silver-Gold-Platinum, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Bronze-Silver-Gold-Platinum
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Diamond-Master-GrandMaster-Professional
Prediction: Diamond-Master-GrandMaster-Professional, Actual: Bronze-Silver-Gold-Platinum
Prediction: Bronze-Silver-Gold-Platinum, Actual: Diamond-Master-GrandMaster-Professional
